In [ ]:
!pip install openai

In [ ]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=ceb8ad29422e735413cfe43502e84c5ab3620199b56e52bd40d68e35cd2af71a
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [ ]:
!pip install gradio

In [ ]:
import openai
import arxiv
import asyncio
import gradio as gr


In [ ]:
from google.colab import userdata

api_key = userdata.get("OPENAI_API_KEY")

openai.api_key = api_key.strip()

In [ ]:
def search_arxiv(topic: str, max_results: int = 5):
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    return list(search.results())


In [ ]:
async def summarize_paper(title: str, summary: str):
    try:
        system_prompt = "You are a helpful assistant that summarizes journal papers."
        user_prompt = f"Summarize the following research journal titled '{title}':\n{summary}\n"

        client = openai.AsyncClient(api_key=api_key.strip())
        response = await client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5,
            max_tokens=500
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"⚠️ Error while summarizing: {e}"


In [ ]:
def research_summary(topic):
    papers = search_arxiv(topic)

    if not papers:
        return "🚫 No relevant journal papers found.", None

    summaries = []
    references = []

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    for paper in papers:
        title = paper.title
        abstract = paper.summary
        link = paper.entry_id

        summary = loop.run_until_complete(summarize_paper(title, abstract))
        summaries.append(f"🔹 {title} ({paper.published.date()}):\n{summary}\n")
        references.append(f"{title} – {link}")

    loop.close()

    full_output = "📚 **Summary of Research Papers:**\n\n" + "\n".join(summaries) + \
                  "\n\n🔗 **References:**\n" + "\n".join(references)

    file_path = "/tmp/research_summary.txt"
    with open(file_path, "w") as f:
        f.write(full_output)

    return full_output, file_path


In [ ]:
iface = gr.Interface(
    fn=research_summary,
    inputs=gr.Textbox(label="Enter Research Topic"),
    outputs=[gr.Textbox(label="Summarized Output", lines=30), gr.File(label="Download Summary")],
    title="📖 AI-Powered Research Summarizer",
    description="Enter any research topic. The system fetches recent ArXiv journal papers and summarizes them using OpenAI."
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5c4a4ca6b66de10e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
